In [1]:
# To do list:
# - Find my capital city co-ordinates
# - Print my country's name and the enemy's

import tailer
import re
import threading
import csv
import time

print("GAME STARTING - TURN 1: ")
player_city = {"X_coords":"0", "Y_coords":"0", "name":""}
enemy_city = {"X_coords":"0", "Y_coords":"0", "name":""}


###############################################################################################################################################################################

def monitor_log_file(log_file_path, patterns, alert_message):
    """
    Monitor a log file for new entries matching any of the specified patterns and alert when such an entry is found.
    Prints the matching string along with the alert message.
    """
    cityname_switch = 0
    citycoords_switch = 0 
    unit_coords = re.compile(r"(\d{3}), LOC_CIVILIZATION([A-Z_]+)NAME, UNIT_([A-Z]*) (\(\d*\)), (\d*), (\d*), (\d*), (\d*)")
    settler_coords = re.compile(r"(\d{3}), LOC_CIVILIZATION([A-Z_]+)NAME, UNIT_SETTLER (\(\d*\)), (\d*), (\d*),")
    player0_name = re.compile(r"Player 0: Civilization - CIVILIZATION_([A-Z_]+)")
    player1_name = re.compile(r"Player 1: Civilization - CIVILIZATION_([A-Z_]+)")

    #while True:
    try:
            # Open the log file and monitor for new lines
            with open(log_file_path, "r") as logfile:
                # Let me know it's found the file
                print(f"{log_file_path} found!")
##
                if log_file_path  == 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\GameCore.log':
                    for line in logfile:
                        if cityname_switch == 0:
                            player0_match = player0_name.search(line)
                            player1_match = player1_name.search(line)

                            if player0_match:
                                player_city['name'] = player0_match.group(1)
                                print(f"You are playing as {player_city['name']}.")
                            
                            if player1_match:
                                enemy_city['name'] = player1_match.group(1)
                                print(f"Up against {enemy_city["name"]}. GOOD LUCK!")
                                print(f"Your X co-ordinates: {player_city['X_coords']} and your Y: {player_city['Y_coords']}.")
                                cityname_switch = 1
#
                # Use tailer.follow to monitor the file for new lines
                #for line in tailer.follow(logfile):
                if log_file_path  == 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\AStar_GC.log':
                    for line in logfile:
                        if citycoords_switch == 0:
                            settler_match = settler_coords.search(line)
                            if settler_match:
                                player_city['X_coords'] = settler_match.group(4)
                                player_city['Y_coords'] = settler_match.group(5)
                                citycoords_switch = 1

                        match = unit_coords.search(line)
                        if match:
                            city_name = match.group(2)
                            unit_name = match.group(3)
                            unit_x_start = int(match.group(5))
                            unit_y_start = int(match.group(6))
                            unit_x_finish = int(match.group(7))
                            unit_y_finish = int(match.group(8))

                            # Set perimeter
                            upper_x_perimeter = int(player_city['X_coords']) + 5
                            lower_x_perimeter = int(player_city['X_coords']) - 5

                            upper_y_perimeter = int(player_city['Y_coords']) + 5
                            lower_y_perimeter = int(player_city['Y_coords']) - 5


                            # Check if finish co-ordinates are within perimeter:
                            if lower_x_perimeter < unit_x_finish < upper_x_perimeter and lower_y_perimeter < unit_y_finish < upper_y_perimeter:
                                # Ensure I don't alert on my own units
                                if city_name != player_city['name']:
                                    print(f"{unit_name} from {city_name} is nearby! ({unit_x_finish}, {unit_y_finish})")
                                    print(player_city['X_coords'])
                                    print(player_city['Y_coords'])


                            

    except FileNotFoundError:
        print(f"File {log_file_path} not found, checking again in 5 seconds.")
        time.sleep(5)  # Wait for some time before checking again, e.g., 10 seconds

###############################################################################################################################################################################

def monitor_csv_file(csv_file_path, alert_message, column_index=0):
    # Compile regex patterns outside the loop for efficiency
    new_barb_unit_pattern = re.compile(r" UNIT_([A-Z]*)") # Barbarians.csv string when a new one is created.
    city_name_pattern = re.compile(r" LOC_CITY_NAME_([A-Z]*)") # AI_CityBuild.csv string when a new unit is created.
    unit_name_pattern = re.compile(r" UNIT_([A-Z]*)") # Barbarians.csv string when a new one is created.
    turn_number = 1

    while True:
        try:
            # Open the CSV file for reading
            with open(csv_file_path, "r") as file:
                # Let me know it's found the file
                print(f"{csv_file_path} found!")

                if csv_file_path == 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\Barbarians.csv':
                            
                        # Find first log of barbarian unit
                        # Skip the header line
                        next(csv.reader(file))
                        # Read the second line
                        second_line = next(csv.reader(file))
                        new_barb_string = new_barb_unit_pattern.search(second_line[2])
                        unit_string = new_barb_string.group(1)
                        print(f"New Barbarian Military Unit spawned: {unit_string} at{second_line[3]}!")
                        # Return to the start of the file to monitor all lines including the first
                        file.seek(0)

                # Use tailer.follow to monitor the file for new lines
                for line in tailer.follow(file):
                    # Process the latest line from the CSV file
                    reader = csv.reader([line])
                    for row in reader:

                        if csv_file_path ==  'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\AI_Planning.csv':
                            time.sleep(0.2)
                            if int(row[0]) > turn_number:
                                turn_number = int(row[0])
                                print(f"\nTURN {turn_number}:\n {player_city['X_coords']}, {player_city['Y_coords']}") 
                    
                        # If a pattern matches, search for the specific unit string
                        # Check for matches with the new pattern in addition to the existing one
                        if csv_file_path == 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\Barbarians.csv':
                            #time.sleep(0.2)
                            new_barb_string = new_barb_unit_pattern.search(row[column_index])
                            new_barb_coords = row[column_index + 1]

                            if new_barb_string:
                                unit_string = new_barb_string.group(1)
                                print(f"{alert_message} {unit_string} at{new_barb_coords}!")  

                        if csv_file_path == 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\AI_CityBuild.csv':
                            #time.sleep(0.2)                    
                            if row[5] == " COMPLETED":
                                new_unit_string = unit_name_pattern.search(row[6])
                                new_unit_city = city_name_pattern.search(row[2])
                                unit_string = new_unit_string.group(1)
                                city_string = new_unit_city.group(1)
                                print(f"{alert_message} {unit_string} in the city of {city_string}!")    
            

        except FileNotFoundError:
            print(f"File {csv_file_path} not found, checking again in 5 seconds.")
            time.sleep(5)  # Wait for some time before checking again, e.g., 10 seconds

        except AttributeError:
            pass

###############################################################################################################################################################################

def start_monitoring(files_info):
    try:
        """
        Start monitoring multiple files in separate threads, allowing for multiple patterns per file.
        Prints the specific string that triggered the alert.
        """
        threads = []

        for file_info in files_info:
            # Extract common parameters
            file_path = file_info['path']
            patterns = file_info['pattern']
            alert_message = file_info['message']

            # Determine the monitoring function and its arguments based on file type
            if file_info.get('type') == 'log':
                thread = threading.Thread(target=monitor_log_file, args=(file_path, patterns, alert_message))
            elif file_info.get('type') == 'csv':
                # Use .get() for 'column_index' to provide a default value if it's not specified
                column_index = file_info.get('column_index', 0)
                thread = threading.Thread(target=monitor_csv_file, args=(file_path, alert_message, column_index))
            else:
                print(f"Unsupported file type for {file_path}")
                continue  # Skip unsupported file types
            
            threads.append(thread)
            thread.start()
    except IndexError:
        pass

###############################################################################################################################################################################

units = [r" UNIT_SPEARMAN",r" UNIT_SCOUT", r" UNIT_WARRIOR", r" UNIT_BARBARIAN_HORSEMAN", r" UNIT_GALLEY", r" UNIT_SETTLER", r" UNIT_BUILDER", r" UNIT_QUADRIREME", r" UNIT_SWORDSMAN", r" UNIT_ARCHERE", r" UNIT_MAN_AT_ARMSE", r" UNIT_CROSSBOWMAN"]                                                                                                                                            ,

# Example usage
files_info = [

    {'type': 'csv', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\AI_CityBuild.csv', 'pattern': units, 'message': "New AI Unit spawned:", 'column_index': 6},

    {'type': 'csv', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\Barbarians.csv', 'pattern': units, 'message': "New Barbarian Military Unit spawned:", 'column_index': 2},

    {'type': 'csv', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\AI_Planning.csv', 'pattern': None, 'message': "New Barbarian Military Unit spawned:", 'column_index': 0},

    {'type': 'log', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\AStar_GC.log', 'pattern': None, 'message': "", 'column_index': 0},

    {'type': 'log', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\GameCore.log', 'pattern': None, 'message': "", 'column_index': 0}

]

start_monitoring(files_info)


GAME STARTING - TURN 1: 
C:\Users\User\AppData\Local\Firaxis Games\Sid Meier's Civilization VI\Logs\AI_CityBuild.csv found!
C:\Users\User\AppData\Local\Firaxis Games\Sid Meier's Civilization VI\Logs\Barbarians.csv found!
New Barbarian Military Unit spawned: SPEARMAN at 3-21!
C:\Users\User\AppData\Local\Firaxis Games\Sid Meier's Civilization VI\Logs\AI_Planning.csv found!
C:\Users\User\AppData\Local\Firaxis Games\Sid Meier's Civilization VI\Logs\AStar_GC.log found!
SCOUT from _BARBARIAN_ is nearby! (9, 16)
5
14
SCOUT from _BARBARIAN_ is nearby! (7, 16)
5
14
SCOUT from _BARBARIAN_ is nearby! (6, 16)
5
14
C:\Users\User\AppData\Local\Firaxis Games\Sid Meier's Civilization VI\Logs\GameCore.log found!
You are playing as MALI.
Up against EGYPT. GOOD LUCK!
Your X co-ordinates: 5 and your Y: 14.
